In [1]:
import datetime as dt
import requests
import pandas as pd
import numpy as np
import Raw_SleeperLeague as rsl
import Raw_SleeperUser as rsu
import Raw_SleeperRosters as rsr
import pyodbc
import sqlalchemy
from To_SQL import SQL_Insert

# Insert User to SQL

In [2]:
user = rsu.User("poulos")
user_df = user.get_user()
# SQL_Insert(user_df, 'users', 'raw_sleeper').insert()

In [3]:
#This will be used in the User League API Call
user_id = str(user_df['user_id'][0])
user_id

'202776677313417216'

# Insert User League to SQL

In [4]:
league = rsl.League(user_id, "2022")
league_users = league.get_all_users()


In [5]:
# Calling the Sleeper API for the all users in a league League. Returns a list of dictionaries.
url = f"https://api.sleeper.app/v1/user/{id}"
user_list = []
for id in [league_users['user_id']][0]:
    user_list.append(requests.get(f"https://api.sleeper.app/v1/user/{id}".format(x=id)).json())

In [6]:
#Loading dictionaries from list into dataframe
df = pd.DataFrame()
for i in user_list:
    df = pd.concat([df, pd.DataFrame.from_dict(i, orient='index').T])

# Insert Results to SQL Table

In [7]:
# SQL_Insert(df, 'users', 'raw_sleeper').insert()


# Getting all Rosters for a League

In [8]:
##Grabs all rosters for the league_id
rosters = rsr.Rosters('poulos', '2022')
roster_list = rosters.get_rosters()

#Loading dictionaries from list into dataframe
roster_df = pd.DataFrame()
for i in roster_list:
    roster_df = pd.concat([roster_df, pd.DataFrame.from_dict(i, orient='index').T])

In [9]:
starters = []
for starter in roster_df['starters'].values:
    for player in starter:
        starters.append(player)
roster_df = roster_df.explode('players')

#Update starters column to Y/N rather than Player ID
roster_df['starters'] = roster_df['players'].apply(lambda x: 'Y' if x in starters else 'N')

roster_df

,taxi,starters,settings,roster_id,reserve,players,player_map,owner_id,metadata,league_id,keepers,co_owners
0,None,N,"{'wins': 5, 'waiver_position': 2, 'waiver_budg...",1,None,1049,None,464228648535191552,"{'p_nick_4018': 'Boyfriend Material', 'p_nick_...",858366433351020544,None,None
0,None,N,"{'wins': 5, 'waiver_position': 2, 'waiver_budg...",1,None,1535,None,464228648535191552,"{'p_nick_4018': 'Boyfriend Material', 'p_nick_...",858366433351020544,None,None
0,None,Y,"{'wins': 5, 'waiver_position': 2, 'waiver_budg...",1,None,1689,None,464228648535191552,"{'p_nick_4018': 'Boyfriend Material', 'p_nick_...",858366433351020544,None,None
0,None,Y,"{'wins': 5, 'waiver_position': 2, 'waiver_budg...",1,None,2505,None,464228648535191552,"{'p_nick_4018': 'Boyfriend Material', 'p_nick_...",858366433351020544,None,None
0,None,Y,"{'wins': 5, 'waiver_position': 2, 'waiver_budg...",1,None,4017,None,464228648535191552,"{'p_nick_4018': 'Boyfriend Material', 'p_nick_...",858366433351020544,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...
0,None,N,"{'wins': 10, 'waiver_position': 8, 'waiver_bud...",10,"[4039, 7588, 8129]",8150,None,202776677313417216,"{'streak': '3W', 'record': 'LWWWWWLLWLWLWWW', ...",858366433351020544,None,None
0,None,Y,"{'wins': 10, 'waiver_position': 8, 'waiver_bud...",10,"[4039, 7588, 8129]",8205,None,202776677313417216,"{'streak': '3W', 'record': 'LWWWWWLLWLWLWWW', ...",858366433351020544,None,None
0,None,N,"{'wins': 10, 'waiver_position': 8, 'waiver_bud...",10,"[4039, 7588, 8129]",DEN,None,202776677313417216,"{'streak': '3W', 'record': 'LWWWWWLLWLWLWWW', ...",858366433351020544,None,None
0,None,Y,"{'wins': 10, 'waiver_position': 8, 'waiver_bud...",10,"[4039, 7588, 8129]",JAX,None,202776677313417216,"{'streak': '3W', 'record': 'LWWWWWLLWLWLWWW', ...",858366433351020544,None,None


In [200]:
# SQL_Insert(roster_df, 'rosters', 'raw_sleeper').insert()
# parsed = x.groupby('roster_id').ratings.apply(lambda x: pd.DataFrame(x.values[0])).reset_index()
x = roster_df[['roster_id', 'settings']]
# x.set_index('roster_id', inplace=True)
# x.pivot(columns='settings')
# x.explode('settings')

# x['settings'] = x['settings'].apply(lambda x: pd.DataFrame.from_dict(x['settings'], orient='index').T)



# x = x.groupby('roster_id').settings.apply(lambda x.values: pd.DataFrame(list(x.values()), columns=x.keys())).reset_index()
# x.set_index('level_1', inplace=True)
# x.get(x['settings'])
# x = x['settings'].reset_index()
# x.apply(lambda x: pd.DataFrame(x['settings'].values(),columns= x.keys()))
pd.map(lambda x: pd.DataFrame(a.values[0].values(), columns=list(x.values[0].keys())))
# x['settings'].values[0].values()


AttributeError: module 'pandas' has no attribute 'map'

In [ ]:

# roster_settings_dicts = roster_df[['settings', 'roster_id']]
# roster_settings_dicts.explode('settings').index('league_id')

# #Create empty dataframe
# roster_settings_df = pd.DataFrame() 
# #Loading dictionaries from list into dataframe
# for i in roster_settings_dicts:
#     # print(i)
#     roster_settings_df = pd.concat([roster_settings_df, pd.DataFrame.from_dict(i, orient='index')].T)
# # roster_settings_df.drop_duplicates(inplace=True)

# roster_settings_dicts

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10 entries, 0 to 0
Data columns (total 21 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   verification     0 non-null      object
 1   username         10 non-null     object
 2   user_id          10 non-null     object
 3   token            0 non-null      object
 4   summoner_region  0 non-null      object
 5   summoner_name    0 non-null      object
 6   solicitable      0 non-null      object
 7   real_name        0 non-null      object
 8   phone            0 non-null      object
 9   pending          0 non-null      object
 10  notifications    0 non-null      object
 11  metadata         0 non-null      object
 12  is_bot           10 non-null     object
 13  email            0 non-null      object
 14  display_name     10 non-null     object
 15  deleted          0 non-null      object
 16  data_updated     0 non-null      object
 17  currencies       0 non-null      objec

In [ ]:
roster_settings_df